In [1]:
# dataset path : 
# https://www.kaggle.com/hsankesara/flickr-image-dataset

In [2]:
import numpy as np
import pandas as pd
import os 
import cv2
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# here we import the resnet 50 model wich predict diffrent 1000 clases from an image
from tensorflow.keras.applications import ResNet50

In [4]:
resnet = ResNet50(include_top=True)

In [5]:
resnet.summary()

In [6]:
# we are interested in the befor last layer it is there where all the featchers are
from keras.models import Model
print(resnet.layers[-2].output.shape)

In [7]:
# here we will take all the model exept the last layer
last_layer = resnet.layers[-2].output
model = Model(inputs=resnet.input, outputs=last_layer)

In [8]:
# this way we can get the path to all the images
images_dir =  "../input/flickr-image-dataset/flickr30k_images/flickr30k_images/"
images = glob(images_dir+'*.jpg')
images[:5]

In [9]:
# lets read the images
featchers_dict = {}
count  = 0
for img in tqdm(images[:5000]):
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # we resize the image to be compatible with resnet
    image = cv2.resize(image, (224,224))
    # our model is expecting 4 dimentional input (batch size) but image.shape is (224,224,3)
    image = image.reshape(1,224,224,3)
    # now lets predict
    prediction = model.predict(image)
    # our predictions are in the form (1,2048)
    prediction = prediction.reshape(2048,)
    image_name = img.split("/")[-1]
    featchers_dict[image_name] = prediction 
    count += 1
#     # since i dont have much computation power i will not use all the images 
#     if count > 2048:
#         break

<h3>here since we have 2048 for each image its too much for calculating the distance between 
the images so we have 2 solution to reduce this number adding a dense layer with less number of outputs 
or using PCA to reduce the dimentionalty</h3>

In [10]:
data = []
for k, v in tqdm(featchers_dict.items()):
    data.append(v)

In [11]:
from sklearn.decomposition import PCA

data = np.array(data)
pca = PCA(n_components=300)
pca.fit(data)

In [12]:
pca_featchers = pca.transform(data)

In [13]:
pca_featchers.shape

In [14]:
for i, (k, _) in enumerate(featchers_dict.items()):
    featchers_dict[k] = pca_featchers[i]
featchers_dict["2715746315.jpg"].shape

In [15]:
# here we will get a random image and calculate the lowest distent images from it
test_images = images[5000:]
def sample_image(test_images):
    sampled_image = np.random.choice(test_images)
    sampled_image = cv2.imread(sampled_image)
    sampled_image = cv2.cvtColor(sampled_image, cv2.COLOR_BGR2RGB)
    sampled_image_proc = cv2.resize(sampled_image, (224,224))
    sampled_image_proc = sampled_image_proc.reshape(1,224,224,3)
    prediction = model.predict(sampled_image_proc)
    prediction = pca.transform(prediction)
    return sampled_image, prediction.reshape(300)

In [16]:
test_image, test_image_pca = sample_image(test_images)

In [20]:
from scipy.spatial import distance
# this is used to calculate distance between vectors
def find_closest(image_pca, featchers_dict):
    closest_images = []
    distances = {}
    for k, v in featchers_dict.items():
        distances[k] = distance.cosine(image_pca, v)
    closest_images = [k for k, v in sorted(distances.items(), key=lambda item: item[1])[:6]]
    return closest_images

In [21]:
colsest_images = find_closest(test_image_pca, featchers_dict)

In [25]:
plt.figure(figsize=(8,6))
plt.imshow(test_image)

plt.figure(figsize=(100, 50))
for i in range(6):
    image = cv2.imread(images_dir + colsest_images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(image)

plt.show()

In [30]:
def test(test_images):
    test_image, test_image_pca = sample_image(test_images)
    colsest_images = find_closest(test_image_pca, featchers_dict)
    plt.figure(figsize=(8,6))
    plt.imshow(test_image)

    plt.figure(figsize=(100, 100))
    for i in range(6):
        image = cv2.imread(images_dir + colsest_images[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax = plt.subplot(3, 20, 40 +i+ 1)
        plt.imshow(image)

    plt.show()

In [37]:
test(test_images)